In [1]:
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
%matplotlib inline

In [2]:
import os
import cv2
from tqdm import tqdm

#### Prepare csv file formats

In [3]:
annotations = pd.DataFrame() # empty for now, will fill later
classes = pd.DataFrame([['nuclei', 0]]) # do not include background

#### Fill out annotations

In [4]:
data_base = '../data/stage1_train'

In [ ]:
counter_all = 0
counter_bad = 0

for img_id in tqdm(os.listdir(data_base)):
    images_folder = '{}/{}/images/'.format(data_base, img_id)
    masks_folder = '{}/{}/masks/'.format(data_base, img_id)
    img_path = '{}/{}'.format(images_folder, os.listdir(images_folder)[0])

    for mask_id in os.listdir(masks_folder):
        counter_all += 1
        mask_path = '{}/{}'.format(masks_folder, mask_id)
        mask_img = cv2.imread(mask_path)
        mask = cv2.imread(mask_path, 0)

        _, contours, _ = cv2.findContours(
            mask,
            cv2.RETR_TREE,
            cv2.CHAIN_APPROX_SIMPLE
        )
        
        pairs = []
        for cnt_num, c in enumerate(contours):
            cnt = contours[cnt_num]
            x, y, w, h = cv2.boundingRect(cnt)
            pairs.append((x, y, x + w, y + h))
            cv2.rectangle(mask_img, (x, y), (x + w, y + h), (255, 0, 0), 2)

        if len(contours) > 1:
#             plt.imshow(mask_img)
#             plt.show()
            counter_bad += 1


 94%|███████████████████████████████████████████████████████████████████████████▌    | 627/664 [01:47<00:02, 12.83it/s]

In [6]:
classes

,0,1
0,nuclei,0
